In [1]:
import cv2
import numpy as np
from skimage.measure import compare_ssim
from match_elements.matching import image_similarity
import matplotlib.pyplot as plt
import detect_compo.lib_ip.ip_preprocessing as pre

In [2]:
# ***********************************
# ****** Test Image Similarity ******
# ***********************************

In [3]:
# img1 = cv2.imread('data/output/clip/android/a0c.jpg')
# img2 = cv2.imread('data/output/clip/ios/i2c.jpg')

img1 = cv2.imread('data/input/A0.jpg')
img2 = cv2.imread('data/input/I0.png')

In [7]:
image_similarity(img1, img2, method='dhash')

0.8125

In [6]:
image_similarity(img1, img2, method='ssim', is_gray=True)

0.7454922569876243

In [5]:
image_similarity(img1, img2, method='surf', draw_match=True, match_distance_ratio=1.3)

0.5427540255413659

In [4]:
image_similarity(img1, img2, method='sift', draw_match=True, match_distance_ratio=1.3)

0.6490653305068413

In [10]:
cv2.imshow('1', img1)
cv2.imshow('2', img2)
cv2.waitKey()
cv2.destroyAllWindows()

In [1]:
# ****************************************
# ****** Test Adaptive Binarization ******
# ****************************************

In [13]:
img = cv2.imread('data/input/I203.png') 
img_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
cv2.imshow('img',cv2.resize(img, (500,800)))
cv2.imshow('gray',cv2.resize(img_gray, (500,800)))

binary = pre.binarization(img, grad_min=4)
cv2.imshow('bin', cv2.resize(binary, (500,800)))
# thresh,img_bin = cv2.threshold(img_gray,10, 255, cv2.THRESH_BINARY|cv2.THRESH_OTSU)
# cv2.imshow('threshold',cv2.resize(img_bin, (500,800)))
# img_ret11 = cv2.adaptiveThreshold(img_gray,  255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,55,0 ) 
# cv2.imshow('adaptiveThreshold',cv2.resize(img_ret11, (500,800)))
img_ret12 = cv2.adaptiveThreshold(img_gray,  255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,55,0 ) 
cv2.imshow('adaptive2', cv2.resize(img_ret12, (500,800)))


cv2.waitKey()
cv2.destroyAllWindows()

In [14]:
# ************************************
# ****** Test Modal Recognition ******
# ************************************

In [11]:
import requests
import base64
from os.path import join as pjoin
import cv2
import time
import json

In [12]:
def text_detection_longce(input_file='data/input/A0001.jpg', output_file='data/output'):
    post_url = "http://61.177.48.150:5222/ocr/recognition_text"
    name = input_file.replace('\\', '/').split('/')[-1][:-4]
    ocr_root = pjoin(output_file, 'ocr')
    img = cv2.imread(input_file)
    start = time.clock()
    retrys = 0

    with open(input_file, "rb") as f:
        base64_img = base64.b64encode(f.read()).decode('utf8')
    form_data = json.dumps({
        "task": "OCR",
        "image": base64_img,
        "type": "base64"
    })
    # print(input_file)
    headers = {
        "Authorization": None,
        "Content-Type": "application/json"
    }
    res_text = list()
    response = requests.post(url=post_url, headers=headers, data=form_data).text
    res = json.loads(response)
    for result in res['data']:
        if 'words' not in result.keys():
            continue
        content = result['words']
        location = result['location']
        res_text.append(Text(id = result['index'], content = content, location = location))
    board = visualize_texts(img, res_text, shown_resize_height=800, show=show, write_path=pjoin(ocr_root, name+'.png'))
    save_detection_json(pjoin(ocr_root, name+'.json'), res_text, img.shape)
    print("[Text Detection Completed in %.3f s] Input: %s Output: %s" % (time.clock() - start, input_file, pjoin(ocr_root, name+'.json')))

In [13]:
text_detection_longce()

NameError: name 'Text' is not defined